In [5]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.malaria import get_confs, get_df, get_ad_token, load_basics
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api

env = Env()
env.read_env('.env-upswell-vlab', override=True)

userinfo, config, db_conf = get_confs(env)

state = CampaignState(
    userinfo.token,
    get_api(env, userinfo.token),
    config.ad_account,
    config.ad_campaign,
)

m = Marketing(state, userinfo, config)


In [3]:
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf
from typing import List



In [4]:
import pandas as pd


def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities


df = pd.read_csv('output/cities.csv')

In [5]:
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]

In [22]:
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

# include entry survey????
        # Targeting.Field.custom_audiences: [{"id": "23845214311790518"}],
        # Targeting.Field.excluded_custom_audiences: [{"id": "23845214312520518"}],

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    id_ = df.disthash.iloc[0]

    return { 'id': id_,
             'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-hindi-hand-black-white', 'vlab-mnm-hindi-grains'],
             'audiences': ["vlab-mnm-kutcha-lookalike"],
             'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-cross-sectional"],
             'quota': quota, 
             'shortcodes': ['baselineeng'],
             'target_questions': [
                 {'ref': 'md:stratumid', 'op': 'equal', 'field': 'response', 'value': id_},
                 {
                     'name': 'occupation',
                     'ref': '4fc929c7-132d-49b1-a164-515e5cc9064f',
                     'op': 'answered',
                     'field': 'response',
                     'value': None
                 },
             ]}


strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 100, df)).tolist()
    

In [23]:
import json
with open('config/test-strata.json', 'w') as f:
    f.write(json.dumps(strata))

In [24]:
from adopt.marketing import load_strata_conf

strata = load_strata_conf('config/test-strata.json')

In [25]:
from adopt.marketing import CreativeConf

creatives = load_typed_json("config/creatives-new.json", List[CreativeConf])

In [ ]:
strata

In [ ]:
from adopt.malaria import hydrate_strata

strata = hydrate_strata(state, strata, creatives)

In [ ]:
strata

In [ ]:
audience_confs = load_typed_json("config/audiences.json", List[AudienceConf])

df = get_df(db_conf, userinfo.survey_user, audience_confs)

In [ ]:
df

In [14]:
from adopt.audiences import hydrate_audiences
from adopt.marketing import manage_audiences


audiences = hydrate_audiences(df, m, audience_confs)



In [ ]:
instructions = manage_audiences(state, audiences)

In [15]:
len(audiences[0].users)

5483

In [ ]:
from adopt.facebook.update import GraphUpdater
import logging

updater = GraphUpdater(state)
for i in instructions[1:2]:
    report = updater.execute(i)
    logging.info(report)

In [29]:
from adopt.clustering import _base_budget

budget = _base_budget(strata, 2000000, 10000)

In [30]:
instructions = m.update_instructions(strata, budget)

INFO:root:Loaded 0 adsets from campaign impact-evaluation-round-B-test


In [ ]:
instructions

In [ ]:
from adopt.facebook.update import GraphUpdater
import logging

updater = GraphUpdater(state)
    
for i in instructions:
    report = updater.execute(i)
    logging.info(report)